In [6]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd

# --- 数据加载和预处理 ---
def load_sonar_data(file_path):
    """
    加载声纳数据集。
    将标签 'M' (雷) 转换为 1, 'R' (岩石) 转换为 -1。
    """
    data = pd.read_csv(file_path, header=None)
    X = data.iloc[:, :-1].values
    y_str = data.iloc[:, -1].values
    y = np.array([1 if label == 'M' else -1 for label in y_str])
    return X, y

# --- RBF 核函数 ---
def rbf_kernel(x1, x2, gamma):
    """
    计算两个样本点之间的RBF核值。
    K(x1, x2) = exp(-gamma * ||x1 - x2||^2)
    """
    return np.exp(-gamma * np.linalg.norm(x1 - x2)**2)

# --- SVM 训练函数 ---
def train_svm(X_train, y_train, C, gamma, tol=1e-5):
    """
    使用SLSQP算法训练SVM。

    参数:
    X_train (np.array): 训练特征数据 (n_samples, n_features)
    y_train (np.array): 训练标签数据 (n_samples,)
    C (float): 惩罚参数
    gamma (float): RBF核的参数
    tol (float): 用于比较浮点数的容差

    返回:
    alpha_sv (np.array): 支持向量的alpha值
    X_sv (np.array): 支持向量的特征
    y_sv (np.array): 支持向量的标签
    b (float): 偏置项
    all_alphas (np.array): 所有训练样本的alpha值
    """
    n_samples = X_train.shape[0]

    # 预计算 Gram 矩阵 K_ij = K(x_i, x_j)
    K_matrix = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(n_samples):
            K_matrix[i, j] = rbf_kernel(X_train[i], X_train[j], gamma)

    # 对偶问题的目标函数 (最小化)
    # L_D = 0.5 * sum_i sum_j alpha_i alpha_j y_i y_j K(x_i, x_j) - sum_i alpha_i
    def objective_func(alpha):
        # y_i y_j K(x_i, x_j) 部分可以写成 (alpha * y).T @ K_matrix @ (alpha * y)
        # 但更直接的方式是使用原始的 alpha 和 K_matrix, 并确保 y_i y_j 在其中
        # term1 = 0.5 * np.sum(np.outer(alpha * y_train, alpha * y_train) * K_matrix) # 这种写法可能不完全对，因为 y_i y_j
        # 我们需要 alpha_i alpha_j y_i y_j K_ij
        # (alpha_i y_i) (alpha_j y_j) K_ij
        # (y_train[:, None] * K_matrix * y_train[None, :]) creates Q_ij = y_i y_j K_ij
        Q_ij = np.outer(y_train, y_train) * K_matrix
        term1 = 0.5 * alpha @ Q_ij @ alpha
        term2 = np.sum(alpha)
        return term1 - term2

    # 目标函数的雅可比矩阵 (梯度)
    # dL_D / d_alpha_k = sum_j alpha_j y_k y_j K(x_k, x_j) - 1
    #                 = y_k * sum_j alpha_j y_j K(x_k, x_j) - 1
    def objective_jac(alpha):
        Q_ij = np.outer(y_train, y_train) * K_matrix
        grad = Q_ij @ alpha - np.ones(n_samples)
        return grad


    # 约束条件
    # 1. sum(alpha_i * y_i) = 0
    constraints = ({'type': 'eq',
                      'fun': lambda alpha: np.dot(alpha, y_train),
                      'jac': lambda alpha: y_train})

    # 边界条件: 0 <= alpha_i <= C
    bounds = [(0, C) for _ in range(n_samples)]

    # 初始猜测
    alpha_init = np.zeros(n_samples)
    # alpha_init = np.random.rand(n_samples) * C * 0.1 # 另一种初始猜测

    # 使用SLSQP求解
    result = minimize(objective_func, alpha_init, jac=objective_jac,
                      method='SLSQP', bounds=bounds, constraints=constraints,
                      tol=1e-6, options={'maxiter': 1000, 'disp': False})

    if not result.success:
        print(f"警告: SLSQP优化未成功收敛: {result.message}")


    all_alphas = result.x

    # 找出支持向量 (alpha > tol)
    sv_indices = np.where(all_alphas > tol)[0]
    alpha_sv = all_alphas[sv_indices]
    X_sv = X_train[sv_indices]
    y_sv = y_train[sv_indices]

    if len(alpha_sv) == 0:
        # print("警告: 没有找到支持向量。可能C值太小或数据特性导致。")
        # 在这种情况下，b和预测可能无意义或需要特殊处理。
        # 为了代码能继续运行，我们设定一些默认值，但这表示模型训练可能有问题。
        return np.array([]), np.array([]), np.array([]), 0.0, all_alphas


    # 计算偏置项 b
    # b = y_k - sum_{i in SV} alpha_i y_i K(x_i, x_k)
    # 选择 0 < alpha_k < C - tol 的支持向量来计算 b，并取平均值
    margin_sv_indices = np.where((all_alphas > tol) & (all_alphas < C - tol))[0]
    b_values = []

    if len(margin_sv_indices) > 0:
        for k_idx in margin_sv_indices:
            # k_idx 是在 all_alphas 中的索引
            # 我们需要用 X_train[k_idx] 和 y_train[k_idx]
            # sum项是对所有支持向量 (alpha_sv, X_sv, y_sv) 计算的
            decision_val_k_no_b = 0
            for i in range(len(alpha_sv)):
                decision_val_k_no_b += alpha_sv[i] * y_sv[i] * rbf_kernel(X_sv[i], X_train[k_idx], gamma)
            b_values.append(y_train[k_idx] - decision_val_k_no_b)
        b = np.mean(b_values) if b_values else 0
    else:
        # 如果没有在边界内的支持向量 (0 < alpha < C)，所有SV的alpha都近似为C
        # 此时，我们需要更鲁棒的 b 计算方法
        # b_upper for y_k = 1, alpha_k = C => f(x_k) + b <= 1 => b <= 1 - f(x_k)
        # b_lower for y_k = -1, alpha_k = C => f(x_k) + b >= -1 => b >= -1 - f(x_k)
        b_upper_candidates = []
        b_lower_candidates = []

        for k_idx in sv_indices: # 遍历所有 alpha_k approx C 的支持向量
            decision_val_k_no_b = 0
            for i in range(len(alpha_sv)): # sum over identified SVs
                decision_val_k_no_b += alpha_sv[i] * y_sv[i] * rbf_kernel(X_sv[i], X_train[k_idx], gamma)

            if y_train[k_idx] == 1:
                b_upper_candidates.append(1 - decision_val_k_no_b)
            else: # y_train[k_idx] == -1
                b_lower_candidates.append(-1 - decision_val_k_no_b)

        b_max_lower = -np.inf
        if b_lower_candidates:
            b_max_lower = np.max(b_lower_candidates)

        b_min_upper = np.inf
        if b_upper_candidates:
            b_min_upper = np.min(b_upper_candidates)

        if b_max_lower > b_min_upper + tol : # 检查是否有矛盾
             # print(f"警告: b_max_lower ({b_max_lower:.4f}) > b_min_upper ({b_min_upper:.4f}). 可能存在数值问题或模型配置问题。")
             # 这种情况下，通常取中点，或者根据具体情况调整。
             # 为了简单，这里取中点，但实际应用中需要注意。
             # 或者，如果所有 SV 都是同一类，则 b 的范围可能是一侧开放的。
             pass # 允许继续，但这是一个警告信号

        if b_max_lower != -np.inf and b_min_upper != np.inf:
            b = (b_max_lower + b_min_upper) / 2.0
        elif b_max_lower != -np.inf:
            b = b_max_lower
        elif b_min_upper != np.inf:
            b = b_min_upper
        else: # 如果都没有（例如只有一个SV），就用那个SV计算
            # print("警告: 计算b时，b_lower 和 b_upper 均无有效值。尝试用第一个SV计算b。")
            if len(sv_indices)>0:
                k_idx_first_sv = sv_indices[0]
                decision_val_first_sv_no_b = 0
                for i in range(len(alpha_sv)):
                    decision_val_first_sv_no_b += alpha_sv[i] * y_sv[i] * rbf_kernel(X_sv[i], X_train[k_idx_first_sv], gamma)
                b = y_train[k_idx_first_sv] - decision_val_first_sv_no_b
            else:
                b = 0 # Fallback, should not happen if SVs exist

    return alpha_sv, X_sv, y_sv, b, all_alphas

# --- SVM 预测函数 ---
def predict_svm(X_eval, alpha_sv, X_sv, y_sv, b, gamma):
    """
    使用训练好的SVM模型进行预测。
    """
    if len(alpha_sv) == 0 : # 没有支持向量
        # print("预测警告：模型没有支持向量，所有预测将基于b值（若b为0，则预测为0）。")
        scores = np.full(X_eval.shape[0], b)
        predictions = np.sign(scores)
        # 处理 np.sign(0) 的情况，可以将其归为一类，例如 +1
        predictions[predictions == 0] = 1
        return predictions, scores

    scores = np.zeros(X_eval.shape[0])
    for i in range(X_eval.shape[0]):
        s = 0
        for j in range(len(alpha_sv)):
            s += alpha_sv[j] * y_sv[j] * rbf_kernel(X_sv[j], X_eval[i], gamma)
        scores[i] = s + b
    predictions = np.sign(scores)
    # 处理 np.sign(0) 的情况，可以将其归为一类，例如 +1
    predictions[predictions == 0] = 1
    return predictions, scores


# --- 主程序 ---
if __name__ == "__main__":
    # 加载数据 [cite: 2]
    file_path = 'sonar.all-data'
    X, y = load_sonar_data(file_path)
    n_total_samples = X.shape[0] # 208

    epsilon_alpha_is_C = 1e-4 # 用于判断 alpha_i 是否等于 C 的容差

    # --- 任务1: C=10, gamma=1 的13次模拟 --- [cite: 4, 5, 6, 7, 8]
    print("--- 任务1: C=10, gamma=1 ---")
    C_val_task1 = 10.0
    gamma_val_task1 = 1.0
    n_simulations = 13

    train_errors_task1 = []
    test_errors_task1 = []
    num_svs_task1 = []
    # “正确分类的离群点”是指那些 alpha_i = C 并且被正确分类的训练样本
    correctly_classified_outliers_task1 = [] # [cite: 8]

    for sim_idx in range(n_simulations):
        # 定义测试集索引 [cite: 7]
        test_indices = np.arange(sim_idx, n_total_samples, n_simulations)
        train_indices = np.array([i for i in range(n_total_samples) if i not in test_indices])

        X_train, y_train = X[train_indices], y[train_indices]
        X_test, y_test = X[test_indices], y[test_indices]

        # 训练SVM
        alpha_sv, X_sv_runtime, y_sv_runtime, b_runtime, all_train_alphas = train_svm(X_train, y_train, C_val_task1, gamma_val_task1, tol=1e-5)

        if len(alpha_sv) == 0: # 如果没有SV，则跳过此轮模拟的统计
            print(f"  模拟 {sim_idx+1}/{n_simulations}: 未找到支持向量，跳过。")
            # 可以选择记录为失败或使用默认错误率（如0.5或1.0）
            train_errors_task1.append(0.5) # 假设错误率
            test_errors_task1.append(0.5)  # 假设错误率
            num_svs_task1.append(0)
            correctly_classified_outliers_task1.append(0)
            continue

        # 训练集错误率
        train_preds, train_scores = predict_svm(X_train, alpha_sv, X_sv_runtime, y_sv_runtime, b_runtime, gamma_val_task1)
        train_error = np.mean(train_preds != y_train)
        train_errors_task1.append(train_error)

        # 测试集错误率
        test_preds, _ = predict_svm(X_test, alpha_sv, X_sv_runtime, y_sv_runtime, b_runtime, gamma_val_task1)
        test_error = np.mean(test_preds != y_test)
        test_errors_task1.append(test_error)

        # 支持向量数量
        num_svs_task1.append(len(alpha_sv))

        # 正确分类的离群点数量
        # 离群点是训练集中 alpha_i 约等于 C 的点
        # all_train_alphas 是对应 X_train 的所有 alpha 值
        outlier_candidate_indices = np.where(np.abs(all_train_alphas - C_val_task1) < epsilon_alpha_is_C)[0]
        count_correct_outliers = 0
        if len(outlier_candidate_indices) > 0:
            # 我们需要对这些离群点候选者进行分类判断
            # train_preds 是对整个训练集的预测，可以直接使用
            for idx_in_train in outlier_candidate_indices:
                # 检查该点是否被正确分类
                if train_preds[idx_in_train] == y_train[idx_in_train]:
                    count_correct_outliers += 1
        correctly_classified_outliers_task1.append(count_correct_outliers)
        print(f"  模拟 {sim_idx+1}/{n_simulations}: 训练误差={train_error:.4f}, 测试误差={test_error:.4f}, SV数={len(alpha_sv)}, 正确离群点数={count_correct_outliers}")


    # 计算均值和标准差 [cite: 6, 8]
    avg_train_error_task1 = np.mean(train_errors_task1)
    std_train_error_task1 = np.std(train_errors_task1)
    avg_test_error_task1 = np.mean(test_errors_task1)
    std_test_error_task1 = np.std(test_errors_task1)
    avg_num_svs_task1 = np.mean(num_svs_task1)
    avg_correct_outliers_task1 = np.mean(correctly_classified_outliers_task1)

    print(f"\n任务1 (C={C_val_task1}, gamma={gamma_val_task1}) 结果:")
    print(f"  平均训练误差: {avg_train_error_task1:.4f} (标准差: {std_train_error_task1:.4f})")
    print(f"  平均测试误差: {avg_test_error_task1:.4f} (标准差: {std_test_error_task1:.4f})")
    print(f"  平均支持向量数量: {avg_num_svs_task1:.2f}")
    print(f"  平均正确分类的离群点数量: {avg_correct_outliers_task1:.2f}")
    print("-" * 30)

    # --- 任务2: 不同C值下的分析 (gamma=1) --- [cite: 9, 10]
    print("\n--- 任务2: 不同C值下的分析 (gamma=1) ---")
    gamma_val_task2 = 1.0
    C_values_task2 = [0.01, 0.1, 1, 10, 50, 100] # 至少5个C值
    results_table_task2 = []

    for C_val in C_values_task2:
        current_C_num_svs = []
        current_C_correct_outliers = []
        current_C_incorrect_outliers = []

        for sim_idx in range(n_simulations):
            test_indices = np.arange(sim_idx, n_total_samples, n_simulations)
            train_indices = np.array([i for i in range(n_total_samples) if i not in test_indices])
            X_train, y_train = X[train_indices], y[train_indices]
            # X_test, y_test = X[test_indices], y[test_indices] # 测试集在此部分不直接用于表格中的指标

            alpha_sv, X_sv_runtime, y_sv_runtime, b_runtime, all_train_alphas = train_svm(X_train, y_train, C_val, gamma_val_task2, tol=1e-5)

            if len(alpha_sv) == 0:
                print(f"  C={C_val}, 模拟 {sim_idx+1}: 未找到支持向量，跳过。")
                current_C_num_svs.append(0)
                current_C_correct_outliers.append(0)
                current_C_incorrect_outliers.append(0)
                continue

            current_C_num_svs.append(len(alpha_sv))

            # 离群点分析 (alpha_i approx C)
            outlier_candidate_indices = np.where(np.abs(all_train_alphas - C_val) < epsilon_alpha_is_C)[0]
            count_correct_outliers_curr_c = 0
            count_incorrect_outliers_curr_c = 0

            if len(outlier_candidate_indices) > 0:
                # 需要对这些离群点候选者进行分类判断
                # 我们需要对训练集进行预测来判断这些 alpha=C 的点是否被正确分类
                train_preds_for_outliers, _ = predict_svm(X_train, alpha_sv, X_sv_runtime, y_sv_runtime, b_runtime, gamma_val_task2)

                for idx_in_train in outlier_candidate_indices:
                    if train_preds_for_outliers[idx_in_train] == y_train[idx_in_train]:
                        count_correct_outliers_curr_c += 1
                    else:
                        count_incorrect_outliers_curr_c += 1
            current_C_correct_outliers.append(count_correct_outliers_curr_c)
            current_C_incorrect_outliers.append(count_incorrect_outliers_curr_c)
            # print(f"  C={C_val}, 模拟 {sim_idx+1}: SVs={len(alpha_sv)}, CorrectOutliers={count_correct_outliers_curr_c}, IncorrectOutliers={count_incorrect_outliers_curr_c}")


        avg_num_svs_curr_c = np.mean(current_C_num_svs)
        avg_correct_outliers_curr_c = np.mean(current_C_correct_outliers)
        avg_incorrect_outliers_curr_c = np.mean(current_C_incorrect_outliers)
        results_table_task2.append({
            "C": C_val,
            "Average number of support vectors": f"{avg_num_svs_curr_c:.2f}",
            "Average number of correct outliers": f"{avg_correct_outliers_curr_c:.2f}",
            "Average number of incorrect outliers": f"{avg_incorrect_outliers_curr_c:.2f}"
        })
        print(f"  C={C_val}: Avg SVs={avg_num_svs_curr_c:.2f}, Avg Correct Outliers={avg_correct_outliers_curr_c:.2f}, Avg Incorrect Outliers={avg_incorrect_outliers_curr_c:.2f}")


    print("\n任务2 表格结果[cite: 10]:")
    df_task2 = pd.DataFrame(results_table_task2)
    print(df_task2.to_string())

    print("\n任务2 趋势分析:")
    print("1. C 与支持向量数量的关系:")
    print("   - 当 C 值较小时，模型对误分类的惩罚较小，倾向于选择一个更宽的间隔（margin），即使这意味着一些点会落入间隔内或被错误分类。这通常导致较少的支持向量，主要是那些定义间隔边界的点。")
    print("   - 当 C 值增大时，模型对误分类的惩罚变大。模型会试图将尽可能多的点正确分类，即使这意味着间隔会变窄。这会导致更多的点成为支持向量，因为模型对数据拟合得更紧密。")
    print("   - 因此，通常情况下，随着 C 的增加，支持向量的数量会增加或保持在一个较高的水平，因为更多的点会影响决策边界的确定。")
    print("\n2. C 与正确/错误分类的离群点数量的关系 (离群点指 alpha_i = C 的点):")
    print("   - 正确分类的离群点 (alpha_i=C, y_i * f(x_i) > 0): 这些是位于间隔边界上或间隔内但被正确分类的点。")
    print("   - 错误分类的离群点 (alpha_i=C, y_i * f(x_i) < 0): 这些是被错误分类的点。")
    print("   - 当 C 值较小时，模型容忍更多的错误和间隔内的点。可能有较多的点其 alpha 达到 C，其中一些可能是被容忍的错误分类（错误离群点），另一些可能是间隔内被正确分类的点（正确离群点）。")
    print("   - 当 C 值增大时，模型对错误的容忍度降低。")
    print("     - 错误分类的离群点数量可能会减少，因为模型努力避免它们。然而，如果数据非常复杂或有噪声，即使C很大，也可能存在一些顽固的错误分类点。")
    print("     - 正确分类的离群点（那些在间隔内但被正确分类的点，alpha=C）的数量可能会变化。如果C很大，模型试图使间隔“干净”，可能会减少这类点。但同时，更多的点可能成为边界上的支持向量（alpha < C）。对于alpha=C的点，它们是那些模型“尽力”也无法使其满足 y_i*f(x_i) >= 1 的点。")
    print("   - 总的来说，随着 C 增加：")
    print("     - 模型对训练数据的拟合程度增加。")
    print("     - 错误分类的离群点数量（alpha=C且错分）一般期望减少，因为惩罚变大了。")
    print("     - 正确分类的离群点数量（alpha=C且正分，但 $y_i f(x_i) \le 1$）的行为可能更复杂，取决于数据集。如果C非常大，模型会尝试将所有点都正确分类并推出间隔，所以理论上这类点（alpha=C）应该主要是那些难以处理的点。")
    print("-" * 30)


    # --- 任务3: 超参数调优 --- [cite: 11]
    print("\n--- 任务3: 超参数调优 ---")
    # 设定gamma和C的候选值范围
    gamma_values_tune = [0.01, 0.1, 0.5, 1, 5, 10]
    C_values_tune = [0.1, 1, 10, 50, 100, 200]

    best_avg_test_error = float('inf')
    best_gamma = None
    best_C = None
    all_tuning_results = []

    print(f"开始超参数调优，Gamma候选: {gamma_values_tune}, C候选: {C_values_tune}")
    total_configs = len(gamma_values_tune) * len(C_values_tune)
    config_count = 0

    for gamma_val in gamma_values_tune:
        for C_val in C_values_tune:
            config_count += 1
            # print(f"  调优进度: {config_count}/{total_configs} (Gamma={gamma_val}, C={C_val})")
            current_config_test_errors = []
            for sim_idx in range(n_simulations):
                test_indices = np.arange(sim_idx, n_total_samples, n_simulations)
                train_indices = np.array([i for i in range(n_total_samples) if i not in test_indices])
                X_train, y_train = X[train_indices], y[train_indices]
                X_test, y_test = X[test_indices], y[test_indices]

                alpha_sv, X_sv_runtime, y_sv_runtime, b_runtime, _ = train_svm(X_train, y_train, C_val, gamma_val, tol=1e-5)

                if len(alpha_sv) == 0:
                    current_config_test_errors.append(0.5) # 假设错误率
                    continue

                test_preds, _ = predict_svm(X_test, alpha_sv, X_sv_runtime, y_sv_runtime, b_runtime, gamma_val)
                test_error = np.mean(test_preds != y_test)
                current_config_test_errors.append(test_error)

            avg_test_error_curr_config = np.mean(current_config_test_errors)
            all_tuning_results.append({'gamma': gamma_val, 'C': C_val, 'avg_test_error': avg_test_error_curr_config})
            print(f"  Gamma={gamma_val}, C={C_val}: 平均测试误差 = {avg_test_error_curr_config:.4f}")

            if avg_test_error_curr_config < best_avg_test_error:
                best_avg_test_error = avg_test_error_curr_config
                best_gamma = gamma_val
                best_C = C_val

    print("\n任务3 超参数调优结果:")
    # 可以选择打印所有调优结果
    # df_tuning = pd.DataFrame(all_tuning_results)
    # print(df_tuning.sort_values(by='avg_test_error').to_string())

    if best_gamma is not None and best_C is not None:
        print(f"  最佳平均测试误差: {best_avg_test_error:.4f}")
        print(f"  对应的最佳 Gamma: {best_gamma}")
        print(f"  对应的最佳 C: {best_C}")
    else:
        print("  未能找到有效的超参数组合（可能所有组合都没有支持向量）。")
    print("-" * 30)

    print("\n代码执行完毕。")

--- 任务1: C=10, gamma=1 ---
  模拟 1/13: 训练误差=0.0000, 测试误差=0.1250, SV数=144, 正确离群点数=0
  模拟 2/13: 训练误差=0.0000, 测试误差=0.0625, SV数=144, 正确离群点数=0
  模拟 3/13: 训练误差=0.0000, 测试误差=0.1875, SV数=143, 正确离群点数=0
  模拟 4/13: 训练误差=0.0000, 测试误差=0.1250, SV数=141, 正确离群点数=0
  模拟 5/13: 训练误差=0.0000, 测试误差=0.1250, SV数=146, 正确离群点数=0
  模拟 6/13: 训练误差=0.0000, 测试误差=0.0625, SV数=145, 正确离群点数=0
  模拟 7/13: 训练误差=0.0000, 测试误差=0.1875, SV数=145, 正确离群点数=0
  模拟 8/13: 训练误差=0.0000, 测试误差=0.3125, SV数=144, 正确离群点数=0
  模拟 9/13: 训练误差=0.0000, 测试误差=0.1250, SV数=141, 正确离群点数=0
  模拟 10/13: 训练误差=0.0000, 测试误差=0.0625, SV数=140, 正确离群点数=0
  模拟 11/13: 训练误差=0.0000, 测试误差=0.1250, SV数=145, 正确离群点数=0
  模拟 12/13: 训练误差=0.0000, 测试误差=0.0000, SV数=139, 正确离群点数=0
  模拟 13/13: 训练误差=0.0000, 测试误差=0.0625, SV数=140, 正确离群点数=0

任务1 (C=10.0, gamma=1.0) 结果:
  平均训练误差: 0.0000 (标准差: 0.0000)
  平均测试误差: 0.1202 (标准差: 0.0754)
  平均支持向量数量: 142.85
  平均正确分类的离群点数量: 0.00
------------------------------

--- 任务2: 不同C值下的分析 (gamma=1) ---


/usr/local/lib/python3.11/dist-packages/scipy/optimize/_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
/usr/local/lib/python3.11/dist-packages/scipy/optimize/_slsqp_py.py:439: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  g = append(wrapped_grad(x), 0.0)


  C=0.01: Avg SVs=184.08, Avg Correct Outliers=84.31, Avg Incorrect Outliers=89.54
  C=0.1: Avg SVs=183.38, Avg Correct Outliers=89.92, Avg Incorrect Outliers=84.31
  C=1: Avg SVs=153.77, Avg Correct Outliers=62.54, Avg Incorrect Outliers=1.77
  C=10: Avg SVs=142.85, Avg Correct Outliers=0.00, Avg Incorrect Outliers=0.00
  C=50: Avg SVs=142.85, Avg Correct Outliers=0.00, Avg Incorrect Outliers=0.00
  C=100: Avg SVs=142.85, Avg Correct Outliers=0.00, Avg Incorrect Outliers=0.00

任务2 表格结果[cite: 10]:
        C Average number of support vectors Average number of correct outliers Average number of incorrect outliers
0    0.01                            184.08                              84.31                                89.54
1    0.10                            183.38                              89.92                                84.31
2    1.00                            153.77                              62.54                                 1.77
3   10.00                        